# Stock price simulation in the Black-Scholes model

In the Black-Scholes world, the stock price follows the SDE:
$
dS = (r − q)Sdt + σ SdW
$

Solution of the SDE:
$
S_t = S_0 \exp \left( \left( r − q − \frac{σ^2}{2} \right) t + σ W_t \right)
$

Discretization:
$
S(t_{k+1}) = S(t_k) \exp \left( \left( r − q − \frac{σ^2}{2} \right) (t_{k+1} − t_k) + σ (W(t_{k+1}) − W(t_k)) \right)
$

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
# Read the data
path = "SP500_03_01_00.xls"
S0 = pd.read_excel(path, "stock")

In [19]:
# Define parameters
S0 = S0['price'].iloc[0] # Initial stock price
K_exotic = 1500  # Strike price
H = 1570  # Barrier level
T_exotic = 47 / 365  # Time to maturity (47 days)
sigma = 0.2228  # Volatility
r_exotic = 0.0603  # Risk-free rate
q = 0.0  # Dividend yield
N = 47  # Number of time steps
M_values = [1000, 10000, 100000]  # Different number of Monte Carlo paths

In [ ]:
def simulate_stock_paths(S0, r, q, sigma,T, N, M):
    dt = 1/365  # Time step size
    stock_paths = np.zeros((M, N + 1))  # Store stock prices for all paths
    stock_paths[:, 0] = S0  # Initial stock price
    
    # Generate random numbers for Brownian motion
    Z = np.random.randn(M, N)
    
    for i in range(1, N + 1):
        stock_paths[:, i] = stock_paths[:, i-1] * np.exp((r - q - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, i-1])
    
    return stock_paths

In [21]:
def price_asian_call(S0, K, r, q, sigma, T, N, M):
    stock_paths = simulate_stock_paths(S0, r, q, sigma, T, N, M)
    average_prices = np.mean(stock_paths[:, 1:], axis=1)  # Average stock price per path
    payoffs = np.maximum(average_prices - K, 0)
    option_price = np.exp(-r * T) * np.mean(payoffs)
    return option_price

In [22]:
def price_up_and_in_put(S0, K, H, r, q, sigma, T, N, M):
    stock_paths = simulate_stock_paths(S0, r, q, sigma, T, N, M)
    
    # Check if barrier was hit for each path
    barrier_hit = np.any(stock_paths[:, 1:] >= H, axis=1)
    payoffs = np.maximum(K - stock_paths[:, -1], 0) * barrier_hit
    option_price = np.exp(-r * T) * np.mean(payoffs)
    return option_price

In [23]:
def price_up_and_out_put(S0, K, H, r, q, sigma, T, N, M):
    stock_paths = simulate_stock_paths(S0, r, q, sigma, T, N, M)
    
    # Check if barrier was hit for each path
    barrier_hit = np.any(stock_paths[:, 1:] >= H, axis=1)
    payoffs = np.maximum(K - stock_paths[:, -1], 0) * (1 - barrier_hit)
    option_price = np.exp(-r * T) * np.mean(payoffs)
    return option_price

In [24]:
# Run simulations for different M values
for M in M_values:
    ac_price = price_asian_call(S0, K_exotic, r_exotic, q, sigma, T_exotic, N, M)
    uibp_price = price_up_and_in_put(S0, K_exotic, H, r_exotic, q, sigma, T_exotic, N, M)
    uobp_price = price_up_and_out_put(S0, K_exotic, H, r_exotic, q, sigma, T_exotic, N, M)
    
    print(f"M = {M}")
    print(f"Asian Call Option Price: {ac_price:.2f}")
    print(f"Up-and-In Put Option Price: {uibp_price:.2f}")
    print(f"Up-and-Out Put Option Price: {uobp_price:.2f}")
    print("-")


M = 1000
Asian Call Option Price: 13.20
Up-and-In Put Option Price: 2.39
Up-and-Out Put Option Price: 65.08
-
M = 10000
Asian Call Option Price: 12.27
Up-and-In Put Option Price: 2.29
Up-and-Out Put Option Price: 62.44
-
M = 100000
Asian Call Option Price: 12.42
Up-and-In Put Option Price: 2.14
Up-and-Out Put Option Price: 63.47
-
